In [291]:
%run 'init.ipynb'
%run 'functions.ipynb'


/userdata/arocchetti/XENONnT_2019-11-04/SS_OuterCryostat/Co60/output_SS_OuterCryostat_Co60_1_Sort.root


In [303]:
date = "XENONnT_2019-10-16"

materials = [
    "SS_OuterCryostat",
    "SS_InnerCryostat",
    "OuterCryostatReflector",
    "SS_BellPlate",
    "SS_BellSideWall",
    "PmtTpc",
    "Copper_TopRing",
    "Copper_LowerRing",
    "Teflon_Pillar_",
    "Copper_FieldGuard_",
    "Copper_FieldShaperRing_",
    "SS_GateRing",
    "SS_AnodeRing",
    "SS_TopMeshRing",
    "SS_CathodeRing",
    "SS_BottomMeshRing",
    "Teflon_BottomTPC",
    "Teflon_TPC",
    ]

isotope_array = ["U238Pb206",
                "K40",
                "Th228Pb208",
                "U235Pb207",
                "Co60",
                "Cs137",]

array = np.arange(1, 2)
print(date)


XENONnT_2019-10-16


In [281]:
# for material in materials:
#     for isotope in isotope_array:
#         for index in array:
#             rootfile = '/userdata/arocchetti/' + str(date) + '/'+ material +'/'+ isotope + '/output_'+ material + '_' + isotope + '_' + str(index) +  '.root'
#             #print(rootfile)
#             data = get_xyz(rootfile)'

In [282]:
 def get_typepri(rootfile):
    f = ROOT.TFile(rootfile)
    tree = f.Get('events/events')
    out=root_numpy.tree2array(tree)
    typepri = np.array([out['typepri'][i] for i in range(len(out['typepri']))])
    f.Close()

In [283]:
#rewrite the function for nSort files 
def get_xyz(rootfile):
    file = uproot.open(rootfile)
    tree = file["events/events"]
   # xp = tree["xp_pri"].array()
    df = tree.pandas.df([ "xpri", "ypri", "zpri", "epri", "ns", "X", "Y", "Z", "Ed", ])
    df.columns = ['xp', 'yp', 'zp_uc', "epri", "ns", "X", "Y", "Z_uc", "Ed"] #rename 
    #Add some variables
    df['rp'] = np.sqrt(df.xp*df.xp+ df.yp*df.yp)
    df['r2p'] = df.rp*df.rp
    df['R'] = np.sqrt(df.X*df.X+ df.Y*df.Y)
    df['R2'] = (df.R*df.R)
    offset = 1488/2
    df['Z'] = df.Z_uc+offset
    df['zp'] = df.zp_uc+ offset
    get_typepri(rootfile)
    df['typepri'] = typepri
    return df

In [284]:
#example
material = "SS_OuterCryostat"
isotope = "Co60"

rootfile = '/userdata/arocchetti/XENONnT_2019-11-04/'+ material + '/' + isotope +'/'+ 'output_'+ material+'_' + isotope +'_1_Sort.root'
rootfile = '/userdata/arocchetti/XENONnT_2019-11-04/SS_OuterCryostat/U238Pb206/output_SS_OuterCryostat_U238Pb206_1_Sort.root'
print(rootfile)
data = get_xyz(rootfile)
print(len(data))

/userdata/arocchetti/XENONnT_2019-11-04/SS_OuterCryostat/U238Pb206/output_SS_OuterCryostat_U238Pb206_1_Sort.root
2465


In [285]:
def print_volume(data):
    plt.hist2d(data.R, data.zp, bins = (300,300), range = ((0, 1000), (-1500, 1700)), 
    norm = LogNorm())
    plt.ylabel('z [mm]')
    plt.xlabel('r [mm]')
    plt.title(material)
    plt.tight_layout()
    plt.title(material)
    plt.show()

def fv(data):
    data['fv-line'] = ((abs(data.Z/634.))**3+abs(data.R/401956.)**3) #add variable
    data = data[data['fv-line']<1]
    data.head(2)
    return data 

In [286]:
# divide the Cryostat
material == "SS_OuterCryostat"
def divide_outercryo(data):
    flange1 = data[(data.R>815) & (data.zp > 300+offset)]
    flange2 = data[(data.R>815) & (data.zp < 300+offset) & (data.zp > -520 + offset)] 
    flange3 = data[(data.R>815) & (data.zp < -520+offset) & (data.zp > -1000 + offset)]
    shell = data[(data.R<815) & (data.zp > -950)]
    elongation = data[(data.R<815) & (data.zp < -950)]
    return flange1, flange2, flange3, shell, elongation
    

flange1, flange2, flange3, shell, elongation = divide_outercryo(data)
flanges = pd.concat([flange1, flange2, flange3])

In [287]:
def calculate_events(data):
    ss = single_scatter(data)
    fv_ = fv(ss)
    en = energy(fv_, 1, 12)
    events_passing = len(en)
    return events_passing

n = calculate_events(elongation)   
print(n)
# material ="OuterCryostat_Elongation":
# "OuterCryostat_flanges":  OuterCryostatShell     parameters = get_parameters(material)

0


In [312]:
material ="OuterCryostat_Elongation"
def calculate_rate(n_events, material, n_gen, m_fv):
    E = 12
    p = get_parameters(material)
    mass = p['mass']
    print("material= ", material)
    print("mass =", parameters['mass'], "gr")
    print("---------------------")
    for isotope in isotope_array:
        print(isotope)
        contamination = p[isotope]
        time = n_gen / (mass * contamination)
        rate = n_events/(time * m_fv * E)
        alpha = mass / (m_fv * E * n_gen)
        err = "err_"+isotope
        err = str(err)
        print("contamination for", isotope, ":", p[isotope], "+-",p[err] )
        error_rate = alpha * np.sqrt((n_events * p[err])**2 + (contamination*np.sqrt(n_events))**2)
        print("---------> Rate ev:", rate, "+-", error_rate)
#         return rate, error_rate

calculate_rate(n, material, 1000, 4000)

material=  OuterCryostat_Elongation
mass = 61500 gr
---------------------
U238Pb206
contamination for U238Pb206 : 4 +- 2
---------> Rate ev: 0.0 +- 0.0
K40
contamination for K40 : 1.4 +- 0.2
---------> Rate ev: 0.0 +- 0.0
Th228Pb208
contamination for Th228Pb208 : 0.57 +- 0.06
---------> Rate ev: 0.0 +- 0.0
U235Pb207
contamination for U235Pb207 : 0.31 +- 0
---------> Rate ev: 0.0 +- 0.0
Co60
contamination for Co60 : 0.61 +- 0.05
---------> Rate ev: 0.0 +- 0.0
Cs137
contamination for Cs137 : 0.034 +- 0.017
---------> Rate ev: 0.0 +- 0.0


In [315]:
material = 'OuterCryostatShell'
calculate_rate(50, material, 1000, 4000)

material=  OuterCryostatShell
mass = 61500 gr
---------------------
U238Pb206
contamination for U238Pb206 : 2.4 +- 0.7
---------> Rate ev: 1.4234999999999998 +- 0.46141922928747775
K40
contamination for K40 : 2.7 +- 0
---------> Rate ev: 1.6014374999999998 +- 0.22647746317928635
Th228Pb208
contamination for Th228Pb208 : 0.36 +- 0
---------> Rate ev: 0.21352500000000002 +- 0.03019699509057151
U235Pb207
contamination for U235Pb207 : 0.11 +- 0.03
---------> Rate ev: 0.06524375 +- 0.020043764051289117
Co60
contamination for Co60 : 9.7 +- 0.8
---------> Rate ev: 5.7533125 +- 0.9418929580653659
Cs137
contamination for Cs137 : 0.64 +- 0
---------> Rate ev: 0.37960000000000005 +- 0.053683546827682685


In [305]:
isotope_array


['U238Pb206', 'K40', 'Th228Pb208', 'U235Pb207', 'Co60', 'Cs137']

In [ ]:
#data_test

In [11]:
#data_test.ns